In [1]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np
import datasets
import os
import io
import ast
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

from datasets import load_dataset
import datasets
import pandas as pd
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from langdetect import detect

from sklearn import model_selection, naive_bayes, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
%matplotlib inline


tfidf_vectorizer = TfidfVectorizer()

os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings

# Disable the specific UserWarning
warnings.filterwarnings("ignore", message="Using the model-agnostic default `max_length`")

import pandas as pd
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt




In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")


tensor([1.], device='mps:0')


In [3]:


processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
model.to(mps_device)


BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-23): 24 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (projection): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((1024,),

# Unique elements in different columns for EDA

In [4]:
metadata_task_unique_class = ['figure question answering','visual question answering','math word problem','geometry problem solving','textbook question answering']
metadata_category_unique_class = ['general-vqa','math-targeted-vqa']
answer_type_unique_class = ['integer','text','float','list']
metadata_context_unique_class = ['scatter plot','synthetic scene','table','geometry diagram','bar chart','abstract scene','function plot','line plot','natural image','puzzle test','scientific figure','pie chart','map chart','medical image','document image','radar chart','heatmap chart','word cloud']
metadata_grade_unique_class = ['daily life','elementary school','high school','college']
metadata_language_unique_class = ['chinese','english', 'persian']

#'metadata_category', 'metadata_task', 'metadata_context', 'metadata_grade', 'metadata_language'


# Blip Models

In [5]:

def BLIP_Generate_Caption(image_data):

    raw_image = image_data.convert('RGB')
    raw_image_tensor = torch.tensor(np.array(raw_image)).permute(2, 0, 1).unsqueeze(0).to(mps_device)

    # conditional image captioning
    text = "a photography of"
    inputs = processor(raw_image_tensor, text, return_tensors="pt")
    inputs = {key: tensor.to(mps_device) for key, tensor in inputs.items()}

    out = model.generate(**inputs)

    # unconditional image captioning
    inputs = processor(raw_image_tensor, return_tensors="pt")
    inputs = {key: tensor.to(mps_device) for key, tensor in inputs.items()}

    out = model.generate(**inputs)
    return (processor.decode(out[0], skip_special_tokens=True))

# Assuming 'decoded_image' contains the PIL image object
# print(BLIP_Generate_Caption(dataset["testmini"][12]['decoded_image']))

def BLIP_Generate_Caption_from_URL(image_url):
    # Fetch image from URL
    response = requests.get(image_url)
    image_data = Image.open(BytesIO(response.content))

    # Convert image data to tensor
    raw_image = image_data.convert('RGB')
    raw_image_tensor = torch.tensor(np.array(raw_image)).permute(2, 0, 1).unsqueeze(0).to(mps_device)

    # conditional image captioning
    text = "a photography of"
    inputs = processor(raw_image_tensor, text, return_tensors="pt")
    inputs = {key: tensor.to(mps_device) for key, tensor in inputs.items()}
    out = model.generate(**inputs)

    # unconditional image captioning
    inputs = processor(raw_image_tensor, return_tensors="pt")
    inputs = {key: tensor.to(mps_device) for key, tensor in inputs.items()}
    out = model.generate(**inputs)

    return (processor.decode(out[0], skip_special_tokens=True))
         


# Text Processor

In [6]:
# Define preprocessing function
def preprocess_text(text):
    text = text.lower()                                             # Make Lower case
    text = re.sub(r'[^\w\s]', '', text)                             # Removes non alphanumeric and spaces
    tokens = word_tokenize(text)                                    # Tokenizing words
    stop_words = set(stopwords.words('english'))                    # Obtaining stop words like 'a', 'the' etc.
    tokens = [word for word in tokens if word not in stop_words]    # Removing stop words
    lemmatizer = WordNetLemmatizer()                                # Obtaining base of words like 'cat' and not 'cats'
    tokens = [lemmatizer.lemmatize(word) for word in tokens]        # lemmatizing each word. 
    preprocessed_text = ' '.join(tokens)                            # Joining and returning
    return preprocessed_text


# Driver Function

In [7]:
def prediction_mode():
    print ("Enter 1 if only the question is passed                  : ")
    print ("Enter 2 if only the image URL is passed                 : ")
    print ("Enter 3 if both Question and Image URL is to be passed  : ")
    print ("Enter 4 to exit program                                 : ")
    choice = int(input("Enter choice :"))
    if (choice not in [1,2,3]):
        print ("Enter a valid choice again: ")
        prediction_mode()
    else :
        return choice

    # print( prediction_mode())


# Function to choose Data Frame:

In [8]:
def df_splitter_and_vectorizer(df):
            X_train, X_test, y_train, y_test = train_test_split(df['preprocessed_text'], 
                                                    df[['answer_type','metadata_language', 'metadata_skills', 'metadata_task', 
                                                        'metadata_category', 'metadata_context', 'metadata_grade',
                                                        'metadata_split', 'metadata_source', 'metadata_img_height', 
                                                        'metadata_img_width']], 
                                                    test_size=0.4, random_state=42)
            
            

            # Fit the vectorizer on the preprocessed text data and transform it
            X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
            X_test_tfidf = tfidf_vectorizer.transform(X_test)

            return X_train_tfidf, X_test_tfidf, y_train, y_test



# Functions for only image promt

In [9]:

def test_text_processor_for_onlyImage(text, answer_type, X_train_tfidf, y_train):
    print (text)
    # Instantiate SVM classifier
    SVM = svm.SVC(C=10, kernel='linear', degree=2, gamma='auto')
    
    # Fit the SVM classifier on the training dataset
    SVM.fit(X_train_tfidf, y_train[answer_type])

    # Preprocess the input text
    preprocessed_text = preprocess_text(text)
    #print("Preprocessed text:", preprocessed_text)

    # Transform preprocessed text using tfidf_vectorizer
    preprocessed_text = tfidf_vectorizer.transform([preprocessed_text])

    # Predict the label
    prediction = SVM.predict(preprocessed_text)

    # Print the prediction
    print("Prediction for "+answer_type+" :", prediction)


            
def predictor_for_URL( URL, X_train_tfidf, y_train):
    print (URL)
    response = requests.get(URL)
    img = Image.open(BytesIO(response.content))
    img.show()
    for answer_type in ['answer_type','metadata_category', 'metadata_task', 'metadata_context', 'metadata_grade', 'metadata_language']:
        test_text_processor_for_onlyImage(BLIP_Generate_Caption_from_URL(URL), answer_type, X_train_tfidf, y_train)
        print ('\n')

# Function for only Question Prompt

In [10]:
def predictor_for_Question( Question, X_train_tfidf, y_train):

    for answer_type in ['answer_type','metadata_category', 'metadata_task', 'metadata_context', 'metadata_grade', 'metadata_language']:
        test_text_processor_for_onlyImage(Question, answer_type, X_train_tfidf, y_train)
        print ('\n')

# Function For Both Image And Question

In [11]:
def predictor_for_Question_and_URL( Question, URL, X_train_tfidf, y_train):
    print (URL)
    response = requests.get(URL)
    img = Image.open(BytesIO(response.content))
    img.show()
    for answer_type in ['answer_type','metadata_category', 'metadata_task', 'metadata_context', 'metadata_grade', 'metadata_language']:
        test_text_processor_for_onlyImage(BLIP_Generate_Caption_from_URL(URL)+" "+Question, answer_type, X_train_tfidf, y_train)
        print ('\n')

### Getting Processed Data

In [15]:

def get_df_paths():

    """

        This Module will get the path of the 3 DATA FRAMES. 
        It will return csv_path which is a list whose each element is path to the dataframe
        
    """
     
    # Get the current directory
    current_directory = os.getcwd()

    # Navigate one folder back to the parent directory
    parent_directory = os.path.dirname(current_directory)
    

    # Navigate to the "data" folder inside the parent directory
    data_directory = os.path.join(parent_directory, "Data")

    # Navigate to the "processed" subfolder inside the "data" folder
    processed_directory = os.path.join(data_directory, "PROCESSED")

    

    # Define the paths to the data files
    csv_file_paths = [
        os.path.join(processed_directory, "df_for_only_question.csv"),
        os.path.join(processed_directory, "df_for_image_captions.csv"),
        os.path.join(processed_directory, "df_for_image_captions+Questions.csv"),
    ]

    return csv_file_paths

csv_file_paths = get_df_paths()


In [16]:
print (os.getcwd())

/Users/psinha/Desktop/Structred Folder/TEST


# Testing


In [17]:
def class_predictor():
    
    choice = prediction_mode()

    if (choice == 1):
        df = pd.read_csv(csv_file_paths[0])
        X_train_tfidf, X_test_tfidf, y_train, y_test = df_splitter_and_vectorizer(df)
        Question = input("Enter Quesion")       
        print ("\n")
        predictor_for_Question( Question, X_train_tfidf, y_train)

    elif (choice == 2):
        df = pd.read_csv(csv_file_paths[1])
        X_train_tfidf, X_test_tfidf, y_train, y_test = df_splitter_and_vectorizer(df)
        print('Enter Image URL (press Enter to skip): ')
        URL = input().strip('"')
        print ("\n")
        predictor_for_URL( URL, X_train_tfidf, y_train)   
        
    elif (choice == 3):
        df = pd.read_csv(csv_file_paths[2])
        X_train_tfidf, X_test_tfidf, y_train, y_test = df_splitter_and_vectorizer(df)
        #print('Enter Image URL (press Enter to skip): ')
        Question = input("Enter Quesion")
        URL = input('Enter Image URL (press Enter to skip): ').strip('"')
        print ("\n")
        predictor_for_Question_and_URL( Question, URL, X_train_tfidf, y_train)


    else:
        exit()        

In [18]:

class_predictor()

# QUESTION : WHAT IS THE SURFACE INTEGRAL OF THE FOLLOWING FIGURE 
# QUESTION : WHAT IS THE AREA OF THE FIGURE SHOWN BELOW. 


# HORSE PULLING A CAR : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSIkJDpcdIMFHRjsgvc2JD0RspkUStSXuoIBQ&usqp=CAU
# CAT GIVING HIGH FIVE TO STAUTE OF LIBERTY : https://randomwordgenerator.com/img/picture-generator/52e1d5424b56aa14f1dc8460962e33791c3ad6e04e50744074267bd69149c7_640.jpg
# DIES IMAGE : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTzqqxGuOfwNRD61afNt4iX0eBmvcZfCWx5Tg&usqp=CAU
# FOOD CHAIN : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSO-8eWbdZ5aEbz1PJ1ryIFYuuG18u8tCc0YXxg33av7SatC5Er8TK9PplEa6IdXsTbQFQ&usqp=CAU


# MAP QUESTION : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSY3NVSNmEeGeEcwByTywefdCi2fncYFkgPcA&usqp=CAU
# PUZZLE PROBLEM : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRpcIq0dBVcbISQ9c9lIcqFtd7OnUy9U5reFg&usqp=CAU  # Slight miss classification because of word cones.


Enter 1 if only the question is passed                  : 
Enter 2 if only the image URL is passed                 : 
Enter 3 if both Question and Image URL is to be passed  : 
Enter 4 to exit program                                 : 


What is India?
Prediction for answer_type : ['float']


What is India?
Prediction for metadata_category : ['general-vqa']


What is India?
Prediction for metadata_task : ['figure question answering']


What is India?
Prediction for metadata_context : ['bar chart']


What is India?
Prediction for metadata_grade : ['daily life']


What is India?
Prediction for metadata_language : ['english']




In [ ]:
# metadata_task_unique_class = ['figure question answering','visual question answering','math word problem','geometry problem solving','textbook question answering']
# metadata_category_unique_class = ['general-vqa','math-targeted-vqa']
# answer_type_unique_class = ['integer','text','float','list']
# metadata_context_unique_class = ['scatter plot','synthetic scene','table','geometry diagram','bar chart','abstract scene','function plot','line plot','natural image','puzzle test','scientific figure','pie chart','map chart','medical image','document image','radar chart','heatmap chart','word cloud']
# metadata_grade_unique_class = ['daily life','elementary school','high school','college']
# metadata_language_unique_class = ['english','chinese']


In [ ]:

def evaluation_of_model(class_name, class_name_unique_class):

    print("Enter 1 if only question given              :")
    print("Enter 2 if only image given                 :")
    print("Enter 3 if both question and image given    :")
    choice = int(input("ENTER YOUR INPUT TYPE          :"))

    if choice == 1:
        df = pd.read_csv('df_for_only_question.csv')

    elif choice == 2:
        df = pd.read_csv('df_for_image_captions.csv')

    elif choice == 3:
        df = pd.read_csv('df_for_image_captions+Questions.csv')

    else:
        print("INVALID CHOICE ")

    # Assuming df_splitter_and_vectorizer and other necessary functions are defined elsewhere
    X_train_tfidf, X_test_tfidf, y_train, y_test = df_splitter_and_vectorizer(df)

    # Fit the training dataset on the SVM classifier
    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train_tfidf, y_train[class_name])

    # Predict the labels on the validation dataset
    predictions_SVM = SVM.predict(X_test_tfidf)

    confusion_matrix = metrics.confusion_matrix(y_test[class_name], predictions_SVM)

    cm_display = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=class_name_unique_class)

    fig, ax = plt.subplots()
    cm_display.plot(ax=ax, xticks_rotation='vertical')  # Rotating x-axis labels vertically
    ax.set_title("Confusion Matrix For "+class_name)  # Setting the title

    # Print classification report
    report = classification_report(y_test[class_name], predictions_SVM, target_names=class_name_unique_class)
    print("\nClassification Report For "+class_name+"\n", report)

    plt.show()

# evaluation_of_model("metadata_task", metadata_task_unique_class)


In [ ]:
evaluation_of_model ("metadata_task", metadata_task_unique_class)

In [ ]:
evaluation_of_model ("metadata_language", metadata_language_unique_class)